### 볼린저밴드 함수화(3단계로 분리)
1. 첫번째 함수 생성(매개변수;데이터프레임 하나)
    - 결측치, 이상치 제거
    - 컬럼 종가만 두고 삭제
    - 이동 평균선, 상단밴드, 하단밴드 파생변수 생성
2. 두번째 함수 생성(매개변수;데이터프레임 하나)
    - 거래내역 파생변수 생성(값은 "")
    - 볼린저밴드 거래내역 추가
    - 거래내역 출력
3. 세번째 함수 생성(매개변수;데이터프레임 하나)
    - 수익률 계산해 수익률 파생변수 생성(기본 값은 "1")
    - 판매한 날의 수익률 변경
    - 누적 수익률 계산해 새로운 파생변수 생성
    - 최종 누적 수익률 출력

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../csv/GM.csv", index_col="Date")

In [ ]:
def bollinger_1(df):
    # df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(axis="columns")]
    # price_df = df[["Adj Close"]]

    price_df = df.loc[(~df.isin((np.nan, np.inf, -np.inf)).any(axis="columns")),["Adj Close"]]


    price_df["center"] = price_df["Adj Close"].rolling(20).mean()
    price_df["ub"] = price_df["center"] + (2 * price_df["Adj Close"].rolling(20).std())
    price_df["lb"] = price_df["center"] - (2 * price_df["Adj Close"].rolling(20).std())

    return price_df

In [ ]:
df = bollinger_1(df)

In [ ]:
def create_band(df, column_name):
    price_df = df.loc[(~df.isin((np.nan, np.inf, -np.inf)).any(axis="columns")),[column_name]]

    price_df["center"] = price_df[column_name].rolling(20).mean()
    price_df["ub"] = price_df["center"] + (2 * price_df[column_name].rolling(20).std())
    price_df["lb"] = price_df["center"] - (2 * price_df[column_name].rolling(20).std())

    return price_df

In [ ]:
df2 = create_band(df2,"Adj Close")

In [ ]:
def bollinger_2(df):
    df["trade"]=""

    for i in df.index:
        if df.loc[i,"Adj Close"] > df.loc[i,"ub"]:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = ""
            else:
                df.loc[i,"trade"] = ""
        
        elif df.loc[i,"Adj Close"] < df.loc[i,"lb"]:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = "buy"
            else:
                df.loc[i,"trade"] = "buy"

        else:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = "buy"
            else:
                df.loc[i,"trade"] = ""

    print("거래내역\n",df["trade"])

    return df

In [ ]:
df = bollinger_2(df)
df

In [ ]:
df["trade"].value_counts()

In [ ]:
def add_trade(df):
    df["trade"]=""

    column_name = df.columns[0]

    for i in df.index:
        if df.loc[i,column_name] > df.loc[i,"ub"]:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = ""
            else:
                df.loc[i,"trade"] = ""
        
        elif df.loc[i,column_name] < df.loc[i,"lb"]:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = "buy"
            else:
                df.loc[i,"trade"] = "buy"

        else:
            if df.shift(1).loc[i,"trade"] == "buy":
                df.loc[i,"trade"] = "buy"
            else:
                df.loc[i,"trade"] = ""

    print("거래내역\n",df["trade"])

    return df

In [ ]:
df2 = add_trade(df2)
df2

In [ ]:
df2["trade"].value_counts()

In [ ]:
def bollinger_3(df):
    rtn = 1.0
    acc_rtn = 1.0
    df['return'] = 1
    buy = 0.0
    sell = 0.0

    for i in df.index:
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, 'Adj Close']
            print('진입일 :', i, '구매 가격 :', buy)
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매 가격 :', sell, '수익율 :', rtn)

        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0

    for i in df.index:
        _rtn = df.loc[i,"return"]
        acc_rtn *= _rtn
        df.loc[i,"acc_rtn"] = acc_rtn

    print("누적 수익률 : ", acc_rtn)

    return

In [ ]:
result = bollinger_3(df)

In [ ]:
def cal_rtn(df):
    rtn = 1.0
    acc_rtn = 1.0
    df['return'] = 1
    buy = 0.0
    sell = 0.0

    column_name = df.columns[0]

    for i in df.index:
        if (df.shift(1).loc[i, 'trade'] == '') and (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, column_name]
            print('진입일 :', i, '구매 가격 :', buy)
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매 가격 :', sell, '수익율 :', rtn)

        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0

    for i in df.index:
        _rtn = df.loc[i,"return"]
        acc_rtn *= _rtn
        df.loc[i,"acc_rtn"] = acc_rtn

    print("누적 수익률 : ", acc_rtn)

    return df

In [ ]:
res = cal_rtn(df2)

In [139]:
import bollinger as bg
import imp

imp.reload(bg)

<module 'bollinger' from 'c:\\Users\\umjiwoo\\OneDrive\\문서\\금융빅데이터 과정\\ubion\\230413\\bollinger.py'>

In [140]:
df = pd.read_csv("../csv/MSFT.csv", index_col="Date")

In [141]:
df = bg.create_band(df,"Adj Close","20100101","20190101")

In [142]:
df = bg.add_trade(df)

거래내역
 Date
2010-01-04       
2010-01-05       
2010-01-06       
2010-01-07       
2010-01-08       
             ... 
2018-12-24    buy
2018-12-26    buy
2018-12-27    buy
2018-12-28    buy
2018-12-31    buy
Name: trade, Length: 2264, dtype: object


In [143]:
df = bg.cal_rtn(df)

진입일 : 2010-05-06 00:00:00 구매 가격 : 23.07136
판매일 : 2010-09-13 00:00:00 판매 가격 : 20.187721 수익율 : 0.875012179602763
진입일 : 2010-11-23 00:00:00 구매 가격 : 20.319853
판매일 : 2011-01-06 00:00:00 판매 가격 : 23.312819 수익율 : 1.1472926994107686
진입일 : 2011-03-16 00:00:00 구매 가격 : 20.171436
판매일 : 2011-04-06 00:00:00 판매 가격 : 21.278055 수익율 : 1.0548606950937949
진입일 : 2011-05-16 00:00:00 구매 가격 : 19.992422
판매일 : 2011-06-27 00:00:00 판매 가격 : 20.639456 수익율 : 1.032363962705469
진입일 : 2011-08-04 00:00:00 구매 가격 : 21.245531
판매일 : 2011-08-31 00:00:00 판매 가격 : 21.923595 수익율 : 1.0319156061573607
진입일 : 2011-11-01 00:00:00 구매 가격 : 21.420837
판매일 : 2012-01-03 00:00:00 판매 가격 : 22.229851 수익율 : 1.037767618510892
진입일 : 2012-04-04 00:00:00 구매 가격 : 26.087456
판매일 : 2012-06-19 00:00:00 판매 가격 : 25.829544 수익율 : 0.9901135626256542
진입일 : 2012-07-12 00:00:00 구매 가격 : 24.087934
판매일 : 2012-09-06 00:00:00 판매 가격 : 26.551155 수익율 : 1.1022595379080664
진입일 : 2012-09-28 00:00:00 구매 가격 : 25.204538
판매일 : 2013-01-25 00:00:00 판매 가격 : 23.806347 수익율 : 0.9445

In [189]:
import bollinger2 as bg2
import imp

imp.reload(bg2)

data = pd.read_csv("../csv/AAPL.csv", index_col="Date")

In [190]:
a =bg2.Invest(data,"Adj Close")

In [191]:
a.col

'Adj Close'

In [192]:
a.create_band("20100101","20150101")

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,NaN,NaN,NaN
2010-01-05,26.829010,NaN,NaN,NaN
2010-01-06,26.402260,NaN,NaN,NaN
2010-01-07,26.353460,NaN,NaN,NaN
2010-01-08,26.528664,NaN,NaN,NaN
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [193]:
a.add_trade()

거래내역
 Date
2010-01-04       
2010-01-05       
2010-01-06       
2010-01-07       
2010-01-08       
             ... 
2014-12-24    buy
2014-12-26    buy
2014-12-29    buy
2014-12-30    buy
2014-12-31    buy
Name: trade, Length: 1258, dtype: object


,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,NaN,NaN,NaN,
2010-01-05,26.829010,NaN,NaN,NaN,
2010-01-06,26.402260,NaN,NaN,NaN,
2010-01-07,26.353460,NaN,NaN,NaN,
2010-01-08,26.528664,NaN,NaN,NaN,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [194]:
a.cal_rtn()

진입일 : 2010-08-24 00:00:00 구매 가격 : 30.026524
판매일 : 2010-09-08 00:00:00 판매 가격 : 32.90366 수익율 : 1.0958198158401553
진입일 : 2011-03-16 00:00:00 구매 가격 : 41.299767
판매일 : 2011-07-01 00:00:00 판매 가격 : 42.957966 수익율 : 1.040150323366231
진입일 : 2011-11-14 00:00:00 구매 가격 : 47.463268
판매일 : 2011-12-27 00:00:00 판매 가격 : 50.876015 수익율 : 1.0719029081604747
진입일 : 2012-04-16 00:00:00 구매 가격 : 72.601524
판매일 : 2012-06-18 00:00:00 판매 가격 : 73.308609 수익율 : 1.0097392583659814
진입일 : 2012-10-08 00:00:00 구매 가격 : 80.207954
판매일 : 2013-07-29 00:00:00 판매 가격 : 57.243137 수익율 : 0.7136840443530077
진입일 : 2013-09-11 00:00:00 구매 가격 : 60.184383
판매일 : 2013-10-18 00:00:00 판매 가격 : 65.48336 수익율 : 1.0880457144505413
진입일 : 2014-01-03 00:00:00 구매 가격 : 70.019096
판매일 : 2014-03-25 00:00:00 판매 가격 : 70.960335 수익율 : 1.0134426042861222
진입일 : 2014-10-15 00:00:00 구매 가격 : 89.842468
판매일 : 2014-10-23 00:00:00 판매 가격 : 96.557182 수익율 : 1.0747387527243797
진입일 : 2014-12-12 00:00:00 구매 가격 : 101.508751
누적 수익률 :  1.0434103630482257
